In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

from networkx.readwrite import json_graph
import json

import pandas

import copy
import random
import csv

import seaborn as sns

from collections import Counter

# Import and Generate Networks

In [ ]:
# MHO
adjmat_MHO  = pd.read_csv("../results/mho_edges.csv")
adjmat_MHO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MHNO
adjmat_MHNO  = pd.read_csv("../results/mhno_edges.csv")
adjmat_MHNO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MUO
adjmat_MUO  = pd.read_csv("../results/muo_edges.csv")
adjmat_MUO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MUNO
adjmat_MUNO  = pd.read_csv("../results/muno_edges.csv")
adjmat_MUNO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

In [ ]:
# add metadata:
# MHO
metadata_MHO  = pd.read_csv("../results/mho_metadata.csv")
metadata_MHO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MHNO
metadata_MHNO  = pd.read_csv("../results/mhno_metadata.csv")
metadata_MHNO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MUO
metadata_MUO  = pd.read_csv("../results/muo_metadata.csv")
metadata_MUO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

# MUNO
metadata_MUNO  = pd.read_csv("../results/muno_metadata.csv")
metadata_MUNO.rename(columns=lambda x: x.replace('.', '|'), inplace=True)

Add relative abundances as attribute:

In [ ]:
def add_abundance(net, metadata):
  abundance_dict = pd.Series(metadata.Abundance.values, index=metadata.Label).to_dict()
  nx.set_node_attributes(net, abundance_dict, 'Abundance')

In [151]:
def create_net(adjmat):
  # Create an empty graph
  tmpG= nx.Graph()

  # Iterate over the rows and columns of the DataFrame to add edges with weights
  for row in range(len(adjmat)):
    # print(adjmat.iloc[row, 0])
    source = adjmat.iloc[row, 0]
    target = adjmat.iloc[row, 1]
    weight = adjmat.iloc[row, 2]
    # Add nodes and edge to the graph
    tmpG.add_edge(source, target, weight=weight)

    # Skip diagonal and zero-weight edges
    if source != target and weight != 0:
        tmpG.add_edge(source, target,
                      weight=weight,
                      invWeight = 1 - weight)
  return(tmpG)

In [152]:
MHO = create_net(adjmat_MHO)
MHNO = create_net(adjmat_MHNO)
MUO = create_net(adjmat_MUO)
MUNO = create_net(adjmat_MUNO)

In [153]:
add_abundance(MHO, metadata_MHO)
add_abundance(MHNO, metadata_MHNO)
add_abundance(MUO, metadata_MUO)
add_abundance(MUNO, metadata_MUNO)

In [154]:
for net in (MHO, MHNO, MUO, MUNO):
  print(net.order(), net.size())

289 800
340 976
326 683
334 760


In [157]:
for net in (MHO, MHNO, MUO, MUNO):
  print('Connected graph:', nx.is_connected(net))

Connected graph: True
Connected graph: True
Connected graph: False
Connected graph: False


In [158]:
def get_connected_graph(net):
  # Get a list of connected components
  components = list(nx.connected_components(net))
  # Find the largest connected component
  largest_component = max(components, key=len)
  # Create a new graph containing only the largest component
  net_largest = net.subgraph(largest_component)
  print('Connected graph:', nx.is_connected(net_largest))
  return(net_largest)


MHO = get_connected_graph(MHO)
MHNO = get_connected_graph(MHNO)
MUO = get_connected_graph(MUO)
MUNO = get_connected_graph(MUNO)

Connected graph: True
Connected graph: True
Connected graph: True
Connected graph: True


In [159]:
def get_pos_neg_edges(net):
  pos_edges_net, neg_edges_net = [], []
  for edge in net.edges(data = True):
    if edge[-1]['weight'] > 0:
      pos_edges_net.append(edge[-1]['weight'])
    else:
      neg_edges_net.append(edge[-1]['weight'])
  print('% positive edge: {0}\n% negative edges: {1}'.
      format(100 * len(pos_edges_net)/net.size(),
             100 * len(neg_edges_net)/net.size()))

print("######### MHO #########")
get_pos_neg_edges(MHO)
print("######### MHNO #########")
get_pos_neg_edges(MHNO)
print("######### MUO #########")
get_pos_neg_edges(MUO)
print("######### MUNO #########")
get_pos_neg_edges(MUNO)

######### MHO #########
% positive edge: 92.875
% negative edges: 7.125
######### MHNO #########
% positive edge: 98.56557377049181
% negative edges: 1.4344262295081966
######### MUO #########
% positive edge: 98.08541973490428
% negative edges: 1.914580265095729
######### MUNO #########
% positive edge: 98.81422924901186
% negative edges: 1.1857707509881423


In [160]:
def get_basic_properties(net):
  print('Number of nodes: {0} \nNumber of edges: {1}'
  .format(net.order(), net.size()))
  print('Graph density: {0}'.format(nx.density(net)))

print("MHO #############"); get_basic_properties(MHO)
print("MHNO ############"); get_basic_properties(MHNO)
print("MUO #############"); get_basic_properties(MUO)
print("MUNO ############"); get_basic_properties(MUNO)

MHO #############
Number of nodes: 289 
Number of edges: 800
Graph density: 0.019223375624759707
MHNO ############
Number of nodes: 340 
Number of edges: 976
Graph density: 0.01693562380704494
MUO #############
Number of nodes: 319 
Number of edges: 679
Graph density: 0.013386960036276888
MUNO ############
Number of nodes: 332 
Number of edges: 759
Graph density: 0.013813562406726604


In [161]:
MHO_names = list(MHO.nodes())
MHNO_names = list(MHNO.nodes())
MUO_names = list(MUO.nodes())
MUNO_names = list(MUNO.nodes())

In [166]:
meandegMHO = sum([i[1] for i in MHO.degree])/MHO.order()
meandegMHNO = sum([i[1] for i in MHNO.degree])/MHNO.order()
meandegMUO = sum([i[1] for i in MUO.degree])/MUO.order()
meandegMUNO = sum([i[1] for i in MUNO.degree])/MUNO.order()
print(meandegMHO, meandegMHNO, meandegMUO, meandegMUNO)

5.536332179930795 5.741176470588235 4.25705329153605 4.572289156626506


# Node Betweenness Values

In [169]:
def get_betw(G):
  dc = nx.degree_centrality(G)
  clsness = nx.closeness_centrality(G)
  btwness_G = nx.betweenness_centrality(G, weight="invWeight")

  print('Centrality (mean):', sum(dc.values())/len(dc))
  print('Closeness (mean):', sum(clsness.values())/len(clsness))
  print('Betweenness (mean):', sum(btwness_G.values())/len(btwness_G))
  return(dc, clsness, btwness_G)

In [170]:
print("MHNO ###########################")
netdict["MHNO"]["centrality"], netdict["MHNO"]["closeness"], netdict["MHNO"]["betweenness"] = get_betw(MHNO)
print("MHO ###########################")
netdict["MHO"]["centrality"], netdict["MHO"]["closeness"], netdict["MHO"]["betweenness"] = get_betw(MHO)
print("MUNO ###########################")
netdict["MUNO"]["centrality"], netdict["MUNO"]["closeness"], netdict["MUNO"]["betweenness"] = get_betw(MUNO)
print("MUO ###########################")
netdict["MUO"]["centrality"], netdict["MUO"]["closeness"], netdict["MUO"]["betweenness"] = get_betw(MUO)

MHNO ###########################
Centrality (mean): 0.01693562380704493
Closeness (mean): 0.25833779826511893
Betweenness (mean): 0.008663664449913597
MHO ###########################
Centrality (mean): 0.019223375624759724
Closeness (mean): 0.2466502923744123
Betweenness (mean): 0.011196946497393785
MUNO ###########################
Centrality (mean): 0.013813562406726642
Closeness (mean): 0.22244194746343451
Betweenness (mean): 0.01105526197070623
MUO ###########################
Centrality (mean): 0.013386960036276915
Closeness (mean): 0.21168131438908902
Betweenness (mean): 0.01221801185268056


In [ ]:
names = ["MHO", "MHNO", "MUO", "MUNO"]
for (i,net) in enumerate([MHO, MHNO, MUO, MUNO]):
  print('#### ', names[i], '###################')
  print('Average shortest path length:', nx.average_shortest_path_length(net, weight = "invWeight"))

####  MHO ###################
Índice de clusterización: 0.15810433319083853
Camino característico: 3.4214858539284863
Máximo k con k-core: 4
####  MHNO ###################
Índice de clusterización: 0.17002110645596438
Camino característico: 3.4202406832363903
Máximo k con k-core: 5
####  MUO ###################
Índice de clusterización: 0.1468026779155306
Camino característico: 4.305762168063188
Máximo k con k-core: 4
####  MUNO ###################
Índice de clusterización: 0.16204644998246528
Camino característico: 4.151702354379257
Máximo k con k-core: 5


# Export dataframes

In [ ]:
networks = {
    "MHNO": MHNO,
    "MHO": MHO,
    "MUNO": MUNO,
    "MUO": MUO
}

all_nodes = set().union(*[G.nodes() for G in networks.values()])
print("Node total: ", len(all_nodes))

metrics = {}

for net_name, G in networks.items():
    metrics[nombre_red] = {
        "degree": dict(G.degree()),
        "betweenness": nx.betweenness_centrality(G),
        "closeness": nx.closeness_centrality(G)
        }

data = []

for node in all_nodes:
    for net_name in networks.keys():
        degree = metrics[net_name]["degree"].get(node, None)
        betweenness = metrics[net_name]["betweenness"].get(node, None)
        closeness = metrics[net_name]["closeness"].get(node, None)

        data.append({
            "node": node,
            "net": net_name,
            "degree": degree,
            "betweenness": betweenness,
            "closeness": closeness
        })

df = pd.DataFrame(data)

Total de nodos:  355


In [ ]:
df.to_csv("../results/network_metrics.csv")

In [ ]:
shortest_path_lengths = {}

for nombre_red, G in redes.items():
    path_lengths = []
    for nodo_origen, longitudes in dict(nx.shortest_path_length(G, weight = "invWeight")).items():
        path_lengths.extend(longitudes.values()) 

    shortest_path_lengths[nombre_red] = path_lengths

In [ ]:
with open("../results/shortest_paths.json", "w") as f:
    json.dump(shortest_path_lengths, f)

# K-Cores

In [ ]:
cores_MHO = nx.core_number(MHO)
cores_MHNO = nx.core_number(MHNO)
cores_MUO = nx.core_number(MUO)
cores_MUNO = nx.core_number(MUNO)

In [ ]:
df = pd.DataFrame(list(cores_MHO.items()), columns=['Node', 'k-core'])
df.to_csv('../results/MHO_kcores.csv', index=False)

df = pd.DataFrame(list(cores_MHNO.items()), columns=['Node', 'k-core'])
df.to_csv('../results/MHNO_kcores.csv', index=False)

df = pd.DataFrame(list(cores_MUO.items()), columns=['Node', 'k-core'])
df.to_csv('../results/MUO_kcores.csv', index=False)

df = pd.DataFrame(list(cores_MUNO.items()), columns=['Node', 'k-core'])
df.to_csv('../results/MUNO_kcores.csv', index=False)